In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.base import BaseEstimator, ClassifierMixin
import scipy.stats as stats

# 🔹 train_test_split을 import 추가
from sklearn.model_selection import train_test_split

# ✅ 문자열 → 숫자로 변환하는 함수 (횟수 데이터 변환)
def convert_count_str(val):
    if pd.isna(val):
        return 0
    val = str(val).strip()
    if "회 이상" in val:
        return 6
    m = re.search(r'(\d+)회?', val)
    return int(m.group(1)) if m else 0

# ✅ 정자, 난자 기증자 나이 매핑
donor_age_mapping = {
    '만20세 이하': 3, '만21-25세': 5, '만26-30세': 4, '만31-35세': 2,
    '만36-40세': 1, '만41-45세': 0, '알 수 없음': 0
}

def convert_donor_age(val):
    if pd.isna(val):
        return np.nan
    return donor_age_mapping.get(str(val).strip(), np.nan)

# ✅ NaN을 문자열 'NaN'으로 변환하는 함수 (카테고리형 변수 처리)
def convert_nan_to_string(df, category_columns):
    df_copy = df.copy()
    for col in category_columns:
        df_copy[col] = df_copy[col].fillna('NaN').astype("category")
    return df_copy

# ✅ 가중치 적용 함수 (모든 시술 유형 반영)
def apply_feature_weights(X, weight_dict):
    X_weighted = X.copy()
    for category, weights in weight_dict.items():  # 모든 시술 유형(IVF, DI 등) 순회
        for column in X.columns:
            if column in weights:  # 해당 카테고리에 가중치가 있으면 적용
                X_weighted[column] *= weights[column]
    return X_weighted

# ✅ 1. 데이터 로드
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

# ✅ 2. 가중치 데이터 로드 (모든 시술 유형 반영)
weight_data = pd.read_csv('og_weighted_hong.csv', encoding='utf-8')
weight_dict = weight_data.set_index("데이터 항목").to_dict()

# ✅ 3. 시술 당시 나이 변환
age_mapping = {
    '만18-34세': 5, '만35-37세': 4, '만38-39세': 3, '만40-42세': 2, '만43-44세': 1, '만45-50세': 0, '알 수 없음': np.nan
}
train['시술 당시 나이'] = train['시술 당시 나이'].map(lambda x: float(age_mapping.get(str(x).strip(), np.nan)))
test['시술 당시 나이'] = test['시술 당시 나이'].map(lambda x: float(age_mapping.get(str(x).strip(), np.nan)))

# ✅ 4. 횟수 관련 컬럼 변환
count_columns = ["총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수",
                 "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수"]

for col in count_columns:
    train[col] = train[col].astype(str).apply(convert_count_str).astype(int)
    test[col] = test[col].astype(str).apply(convert_count_str).astype(int)

# ✅ 5. 난자/정자 기증자 나이 변환
train['난자 기증자 나이'] = train['난자 기증자 나이'].astype(str).apply(convert_donor_age)
test['난자 기증자 나이'] = test['난자 기증자 나이'].astype(str).apply(convert_donor_age)
train['정자 기증자 나이'] = train['정자 기증자 나이'].astype(str).apply(convert_donor_age)
test['정자 기증자 나이'] = test['정자 기증자 나이'].astype(str).apply(convert_donor_age)

# ✅ 6. Feature 가중치 적용
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

X_weighted = apply_feature_weights(X, weight_dict)
X_test_weighted = apply_feature_weights(test, weight_dict)

# ✅ 7. 데이터 불균형 처리 (임신 성공 여부 기준)
undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersample.fit_resample(X_weighted, y)

# ✅ 8. 범주형 변수 변환
category_columns = ["시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형",
                    "배아 생성 주요 이유", "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이"]

X_resampled = convert_nan_to_string(X_resampled, category_columns)
X_test_weighted = convert_nan_to_string(X_test_weighted, category_columns)

# ✅ 9. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled,
                                                  test_size=0.2, random_state=42, stratify=y_resampled)



In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.base import BaseEstimator, ClassifierMixin
import scipy.stats as stats

# 🔹 train_test_split을 import 추가
from sklearn.model_selection import train_test_split

In [ ]:


#  1. 데이터 로드 및 전처리
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

# ✅ train_test_split 오류 해결
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 데이터 분할 (수정된 부분)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# ✅ 모델 구성
stack_clf = StackingClassifier(
    estimators=[
        ('xgb', XGBClassifier(tree_method='gpu_hist', enable_categorical=True, random_state=42)),
        ('lgbm', LGBMClassifier(n_jobs=1, random_state=42, verbose=-1)),
        ('cat', CatBoostClassifier(task_type='GPU', verbose=0, cat_features=[X_resampled.columns.get_loc(c) for c in category_columns]))
    ],
    final_estimator=Pipeline([
        ('scaler', RobustScaler()),
        ('lr', LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear'))
    ]),
    cv=3,
    n_jobs=1
)

# ✅ 모델 학습
stack_clf.fit(X_train, y_train)

# ✅ 검증 데이터 평가
y_val_pred = stack_clf.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f"Validation ROC AUC: {roc_auc:.5f}")

# ✅ 테스트 데이터 예측
pred_proba = stack_clf.predict_proba(X_test_weighted)[:, 1]

# ✅ 결과 저장
submission = pd.DataFrame({'ID': [f"TEST_{i:05d}" for i in range(len(test))],'probability': pred_proba})
submission.to_csv('/content/drive/MyDrive/aimers/submit/final_model_submit.csv', index=False)
print("제출 파일 생성 완료")


NameError: name 'X_resampled' is not defined